<a href="https://colab.research.google.com/github/Braafisch/ASD-Assignments/blob/main/asd_assignment3/asd_assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vorlesung Automones Fahren, HS Esslingen, Thao Dang

# Spurverlaufsschätzung

## Vorbereitungen

In diesem Programmierbeispiel lernen Sie eine einfache Spurverlaufsschätzung basierend auf einem Kleinste-Quadrate-Schätzer (oder auch Least-Squares(LS)-Schätzer) kennen. In der nächsten Übung werden Sie diesen Schätzer dann verbessern, indem Sie robuste Schätzverfahren implementieren.

Laden der Standard-Bibliotheken:

In [ ]:
import cv2
import numpy as np
import requests
import shutil
from matplotlib import pyplot as plt
%matplotlib inline

Für diese Aufgaben wurden Bilder aus der [Kitti Datenbank](http://www.cvlibs.net/datasets/kitti/), einem frei zugänglichen Benchmark-Dataset für Computer Vision, verwendet. Aus diesen wurden Spurmarkierung extrahiert, rlevante Markierungspunkte  und die Position der Spurmarkierungen in ein DIN70000 Fahrzeugkoordinatensystem transformiert.

![markings](https://drive.google.com/uc?id=1UPhB_8l_5rOqDUWv5xluQrzx5fpTIPY1)

Diese Punkte werden im folgenden geladen.

In [ ]:
# Upload images - Colab ONLY!
# pts_left.npy, pts_right.npy

url = 'https://raw.githubusercontent.com/Braafisch/ASD-Assignments/main/asd_assignment3/'
response = requests.get(url + 'pts_left.npy', allow_redirects = True, stream = True).raw

with open('pts_left.npy', 'wb') as fin:
    shutil.copyfileobj(response, fin)

response = requests.get(url + 'pts_right.npy', allow_redirects = True, stream = True).raw

with open('pts_right.npy', 'wb') as fin:
    shutil.copyfileobj(response, fin)

In [ ]:
lane_left = np.load('pts_left.npy')
lane_right = np.load('pts_right.npy')

plt.title('measurements')
plt.plot(lane_left[:, 1], lane_left[:, 0], 'r.')
plt.plot(lane_right[:, 1], lane_right[:, 0], 'g.')
plt.xlim(30, -30)
plt.xlabel('Y [m]')
plt.ylim(0, 60)
plt.ylabel('X [m]')
plt.grid(True);

## 1. Kleinste-Quadrate-Schätzung

Das in diesem Beispiel verwendete geometrische Spurmodell ist eine vereinfachte Version des der [Dissertation "Dreidimensionale Straßenmodelle für Fahrerassistenzsysteme auf Landstraßen"](https://publikationen.bibliothek.kit.edu/1000030918) vorgestellten Modells: 

![Spurmodell.png](https://drive.google.com/uc?id=1X4wEJQ3h5J_yW-ce2NhzKLBVrcIU31MJ)

Die Parametervektor zur Beschreibung dieser Spur lautet 

\begin{equation}
\mathbf{z} = \left[ W, Y_{\mbox{offset}}, \Delta \phi, c_0 \right]^T, \tag{1}
\end{equation}

wobei W die Strassenbreite, $Y_{\mbox{offset}}$ den Querversatz des Fahrzeugs zur Spurmitte, $\Delta \phi$ den Winkel zur Spur und $c_0$ einen Parabelkoeffizienten bezeichnet.

Mit Hilfe dieser Parameter kann die Position eines Punktes der linken Fahrbahnberandung näherungsweise durch

\begin{equation} 
Y_L = \frac{1}{2} W - Y_{\mbox{offset}} - X_L \Delta \phi + \frac{1}{2} c_0 X_L^2 \tag{2}
\end{equation}

angegeben werden. Für die rechte Fahrbahnberandung gilt analog:

\begin{equation} 
Y_R = - \frac{1}{2} W - Y_{\mbox{offset}} - X_R \Delta \phi + \frac{1}{2} c_0 X_R^2 \tag{3}
\end{equation}

Diese Gleichungen sind die Grundlage der Modellgleichung für die Kleinste-Quadrate-Schätzung der Spurparameter.

**Modellgleichung**

Gegeben seien zunächst mehrere Berandungspunkte $\mathbf{p}_L$ und $\mathbf{p}_R$ (in DIN70000 Koordinaten) der linken bzw. rechten Spurberandung:

\begin{equation}
\mathbf{p}_L = \left[ \begin{array}{cc} 
    X_{L,1} & Y_{L,1} \\
    \vdots & \vdots \\
    X_{L,N} & Y_{L,N}
\end{array} \right] \;, \;\;\;\;
\mathbf{p}_R = \left[ \begin{array}{cc} 
    X_{R,1} & Y_{R,1} \\
    \vdots & \vdots \\
    X_{R,N} & Y_{R,M}
\end{array} \right]
\end{equation}

Bestimmen Sie (in Anlehnung an die Vorlesungsunterlagen und Gln. (2) und (3)) die Designmatrix $\mathbf{H}$ und den Beobachtungsvektor $\mathbf{y}$, sodass gilt:

\begin{equation}
\mathbf{y} = \left[ \begin{array}{c} 
    Y_{L,1} \\
    \vdots \\
    Y_{L,N} \\
    Y_{R,1} \\
    \vdots \\
    Y_{R,M}
\end{array} \right] = \mathbf{H} \, \mathbf{z} + \mathbf{e} \tag{4}
\end{equation}


**Berechnung der Spurparameter**

Bestimmen Sie die optimalen Spurparameter $\mathbf{\hat{z}}$ nach der Methode der kleinsten Parameter, d.h.

\begin{equation}
\mathbf{\hat{z}} = \left( \mathbf{H}^T \mathbf{H} \right)^{-1} \mathbf{H}^T \, \mathbf{y} \tag{5}
\end{equation}


Vervollständigen Sie die Funktion ``LS_lane_fit(pL, pR)``, welche aus den Berandungspunkten $p_L$ und $p_R$ sowie den Gln (4) und (5) die optimalen Spurparameter berechnet und zurückgibt. Vergleichen Sie Ihr Ergebnis mit den wahren Werten von $\mathbf{\hat{z}}$.

**Hinweis:** Zwei ``numpy``-Befehle, die dazu nützlich sein könnten:
1. ``np.dot(A, B)`` berechnet das Matrixprodukt $\mathbf{A} \cdot \mathbf{B}$.
2. ``np.linalg.pinv(H)`` berechnet die Pseudoinverse $\left( \mathbf{H}^T \mathbf{H} \right)^{-1} \mathbf{H}^T$ von $\mathbf{H}$.  

In [ ]:
pL = np.array([[27.47, 1.57], [20.50, 1.79], [6.77, 2.06]])
pR = np.array([[44.34, -2.99], [22.89, -2.03], [7.38, -1.72]])


def LS_lane_fit(pL, pR):
    H = np.zeros((pL.shape[0]+pR.shape[0], 4)) # design matrix
    Y = np.zeros((pL.shape[0]+pR.shape[0], 1)) # noisy observations

    ## HIER CODE EINFUEGEN
    Y = np.asmatrix(np.append(pL[:,1], pR[:,1], axis= 0)).T
    
    H[:, 0] = np.append(np.full(pL.shape[0], 0.5), np.full(pR.shape[0], -0.5), axis= 0)
    H[:, 1] = np.full(H.shape[0], -1)
    H[:, 2] = -np.append(pL[:,0], pR[:,0], axis= 0)
    H[:, 3] = 0.5*np.append(pL[:,0], pR[:,0], axis= 0)**2

    Z = np.dot(np.linalg.pinv(H), Y)
    
    ## EIGENER CODE ENDE
    
    return Z


Z = LS_lane_fit(pL, pR)
print('Estimated lane parameters:')
print(Z.T)
print('Expected lane parameters:')
print('[[ 3.76145399e+00 -2.11954070e-01  1.38925253e-03 -1.28124172e-03]]')

## 2. Visualisierung der Spur

Um den Spurverlauf zu visualisieren, können aus Gln. (2) und (3)  für gegebene Spurparameter $\mathbf{z}$ und gegebene Entfernungen $\mathbf{X}_{pred}$ die erwarteten Querablagen  $\mathbf{Y}_{L,pred}$ und  $\mathbf{Y}_{R,pred}$ berechnet werden. 

Ergänzen Sie dazu die Funktion ```LS_lane_compute(Z, maxDist=60, step=0.5)```. Eingangsparameter sind die Spurparameter ``Z``, sowie die maximal darzustellende Entfernung ``maxDist`` sowie die Schrittweite in X-Richtung ``step`` (beide in Metern).

In [ ]:
def LS_lane_compute(Z, maxDist=60, step=0.5):
    x_pred = np.arange(0, maxDist, step)
    yl_pred = np.zeros_like(x_pred)
    yr_pred = np.zeros_like(x_pred)

    ## HIER CODE EINFUEGEN
    Hl = np.zeros((x_pred.shape[0], 4))
    Hl[:, 0] = np.full(x_pred.shape, 0.5)
    Hl[:, 1] = np.full(x_pred.shape, -1)
    Hl[:, 2] = -x_pred
    Hl[:, 3] = 0.5*x_pred**2
    yl_pred = np.dot(Hl, Z)

    Hr = np.zeros((x_pred.shape[0], 4))
    Hr[:, 0] = np.full(x_pred.shape, -0.5)
    Hr[:, 1] = np.full(x_pred.shape, -1)
    Hr[:, 2] = -x_pred
    Hr[:, 3] = 0.5*x_pred**2
    yr_pred = np.dot(Hr, Z)
    ## EIGENER CODE ENDE
    
    return (x_pred, yl_pred, yr_pred)


x_pred, yl_pred, yr_pred = LS_lane_compute(Z)
    
plt.title('measurements')
plt.plot(pL[:, 1], pL[:, 0], 'rs')
plt.plot(pR[:, 1], pR[:, 0], 'gs')
plt.plot(yl_pred, x_pred, 'b')
plt.plot(yr_pred, x_pred, 'c')
plt.xlim(30, -30)
plt.xlabel('Y [m]')
plt.ylim(0, 60)
plt.ylabel('X [m]')
plt.grid(True);

## 3. Anwendung auf extrahierte Markierungspositionen

Damit sind alle Funktionen für die Kleinste-Quadrate-Berechnung sowie die Visualisierung des Spurverlaufs vorhanden.

Wenden Sie Ihren Code auf die gegebenen Realdaten an und visualisieren Sie die Messungen und den ermittelten Spurverlauf.

In [ ]:
Z = LS_lane_fit(lane_left, lane_right)

x_pred, yl_pred, yr_pred = LS_lane_compute(Z)

plt.title('measurements')
plt.plot(lane_left[:, 1], lane_left[:, 0], 'ro', label='left lane pts')
plt.plot(lane_right[:, 1], lane_right[:, 0], 'gs', label='right lane pts')
plt.legend()
plt.plot(yl_pred, x_pred, 'b')
plt.plot(yr_pred, x_pred, 'c')
plt.xlim(30, -30)
plt.xlabel('Y [m]')
plt.ylim(0, 60)
plt.ylabel('X [m]')
plt.grid(True);

## 4. Residuenanalyse

Berechnen Sie nun die Residuen der Spurverlaufsschätzung. Was fällt Ihnen dabei auf?

In [ ]:
def create_H(x_lane_left,x_lane_rigth):
  length_left = x_lane_left.shape[0]
  length_rigth = x_lane_rigth.shape[0]
  H = np.zeros((length_left + length_rigth, 4))

  H[:, 0] = np.append(np.full(x_lane_left.shape, 0.5), np.full(x_lane_rigth.shape, -0.5), axis= 0)
  H[:, 1] = np.full(H.shape[0], -1)
  H[:, 2] = -np.append(x_lane_left, x_lane_rigth, axis= 0)
  H[:, 3] = 0.5*np.append(x_lane_left, x_lane_rigth, axis= 0)**2
  return H

H = create_H(lane_left[:,0], lane_right[:,0])                          # design matrix
Y = np.asmatrix(np.append(lane_left[:,1], lane_right[:,1], axis= 0)).T # noisy observations

res = np.dot(H, Z) - Y

plt.hist(res, 100);